
Advertiser spend and ROI metrics
  


Given the following two tables:
◦Advertiser info table with columns advertiser_id, ad_id and spend, where spend is what the advertiser paid for this ad.
◦Ad info table with columns ad_id, user_id and price, where price is how much the user_id spend through this ad., assuming all prices > 0.

Questions
◦The fraction of advertisers has at least one conversion.
◦What metrics would you show to advertisers.


In [1]:
import pandas as pd
import pandasql as pdsql

In [61]:
advertiser_info = pd.read_csv("advertiser_info.csv")
ad_info = pd.read_csv("ad_info.csv")

In [11]:
pysql = lambda q: pdsql.sqldf(q, globals())

In [62]:
sql = """
        select cast(sum(case when convert > 0 then 1 else 0 end) as float)/ cast(count(distinct advertiser_id) as float) as fraction
        from
        (
        select 
            advertiser_id
            , sum(case when ad.ad_id is not null then 1 else 0 end) as convert 
        from advertiser_info adv
        left join ad_info ad
        on adv.ad_id = ad.ad_id
        group by advertiser_id)
        """
df = pysql(sql)
df

,fraction
0,0.666667


In [63]:
# With conversion
conversion = advertiser_info.merge(ad_info, left_on = "ad_id",right_on = "ad_id", how = "inner")
float(conversion["advertiser_id"].nunique())/advertiser_info["advertiser_id"].nunique()

0.6666666666666666

In [64]:
# ROI
sql_ROI = """
            select 
                advertiser_id
                , sum(case when adv.spend is null then 0 else adv.spend end) as tot_spend
                , sum(case when ad.price is null then 0 else ad.price end) as tot_return
                , (cast(sum(case when ad.price is null then 0 else ad.price end) as float)/ cast(sum(case when adv.spend is null then 0 else adv.spend end) as float) -1) as ROI 
            from advertiser_info adv
            left join ad_info ad
            on adv.ad_id = ad.ad_id
            group by advertiser_id
            """
df2 = pysql(sql_ROI)

In [65]:
df2

,advertiser_id,tot_spend,tot_return,ROI
0,1,2000.0,1070,-0.465000
1,2,1000.0,0,-1.000000
2,3,6000.0,4000,-0.333333


In [86]:
import numpy as np
adv_all = advertiser_info.merge(ad_info, left_on = "ad_id", right_on = "ad_id", how = "left", indicator = True)
adv_all["spend"] = adv_all["spend"].fillna(0)
adv_all["price"] = adv_all["price"].fillna(0)
ROI = adv_all[["advertiser_id","spend","price"]].groupby("advertiser_id").sum().reset_index()
ROI["ROI"] = ROI["price"].astype(float)/ ROI["spend"].astype(float) - 1.0

In [88]:
ROI2 = adv_all.groupby("advertiser_id").agg({"spend": np.sum, "price": np.sum}).reset_index()
ROI2

,advertiser_id,price,spend
0,1,1070.0,2000.0
1,2,0.0,1000.0
2,3,4000.0,6000.0


In [105]:
ROI3 = adv_all.pivot_table(index = "advertiser_id", values = ["price","spend"], aggfunc = np.sum).reset_index()
ROI3

,advertiser_id,price,spend
0,1,1070.0,2000.0
1,2,0.0,1000.0
2,3,4000.0,6000.0


In [83]:
ROI.dtypes

advertiser_id      int64
spend            float64
price            float64
%Return          float64
dtype: object